# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,letterkenny,54.9500,-7.7333,15.45,72,63,5.36,IE,1724859582
1,1,pitimbu,-7.4706,-34.8086,26.57,68,43,8.05,BR,1724859583
2,2,turpan,42.9333,89.1667,36.00,22,4,0.98,CN,1724859585
3,3,bethel,41.3712,-73.4140,30.03,72,0,2.06,US,1724859044
4,4,yellowknife,62.4560,-114.3525,14.67,89,75,4.63,CA,1724859312


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = 'Country'
)

# Display the map
cities_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp = city_data_df.loc[(city_data_df['Max Temp']>21) | (city_data_df['Max Temp']<27),:]
ideal_wind = ideal_temp.loc[ideal_temp['Wind Speed']<4.5,:]
ideal_cloud = ideal_wind.loc[ideal_wind['Cloudiness']==0,:]

ideal_cloud

# Drop any rows with null values
ideal_city = ideal_cloud.dropna()

# Display sample data
ideal_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,bethel,41.3712,-73.4140,30.03,72,0,2.06,US,1724859044
10,10,petropavlovsk-kamchatsky,53.0452,158.6483,10.33,82,0,4.00,RU,1724859237
76,76,arkhangel'sk,64.5401,40.5433,16.06,63,0,4.00,RU,1724859670
105,105,mazamari,-11.3258,-74.5308,32.07,35,0,1.56,PE,1724859704
111,111,huntingburg,38.2989,-86.9550,31.05,58,0,2.57,US,1724859710
130,130,kone,-21.0595,164.8658,17.56,86,0,2.44,NC,1724859310
140,140,charters towers,-20.1000,146.2667,16.77,94,0,2.62,AU,1724859742
143,143,jurmala,56.9680,23.7704,22.01,60,0,4.12,LV,1724859746
146,146,hamilton,39.1834,-84.5333,31.88,60,0,3.60,US,1724858974
156,156,maun,-19.9833,23.4167,33.98,11,0,2.57,BW,1724859760


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city.loc[:,['City', 'Country', 'Lat', 'Lng', 'Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,bethel,US,41.3712,-73.4140,72,
10,petropavlovsk-kamchatsky,RU,53.0452,158.6483,82,
76,arkhangel'sk,RU,64.5401,40.5433,63,
105,mazamari,PE,-11.3258,-74.5308,35,
111,huntingburg,US,38.2989,-86.9550,58,
130,kone,NC,-21.0595,164.8658,86,
140,charters towers,AU,-20.1000,146.2667,94,
143,jurmala,LV,56.9680,23.7704,60,
146,hamilton,US,39.1834,-84.5333,60,
156,maun,BW,-19.9833,23.4167,11,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
arkhangel'sk - nearest hotel: Серафима
mazamari - nearest hotel: Hospedaje El Conquistador
huntingburg - nearest hotel: Quality Inn
kone - nearest hotel: Pacifik Appartels
charters towers - nearest hotel: No hotel found
jurmala - nearest hotel: Kurši
hamilton - nearest hotel: North Vista Manor
maun - nearest hotel: Center Lodge Conference Center
cabo san lucas - nearest hotel: Comfort Rooms
tsiombe - nearest hotel: No hotel found
rouissat - nearest hotel: فندق الأروي الذهبي
aripuana - nearest hotel: No hotel found
guiren - nearest hotel: No hotel found
spring creek - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
laramie - nearest hotel: Laramie Valley Inn
harrisburg - nearest hotel: Alva Hotel and Restaurant
vennesla - nearest hotel: No hotel found
hassi messaoud - nearest hotel: AMC
clifton - nearest hotel: La Quinta Inn & Suites Clifton

,City,Country,Lat,Lng,Humidity,Hotel Name
3,bethel,US,41.3712,-73.4140,72,Hampton Inn Danbury
10,petropavlovsk-kamchatsky,RU,53.0452,158.6483,82,ООО Постоялый двор
76,arkhangel'sk,RU,64.5401,40.5433,63,Серафима
105,mazamari,PE,-11.3258,-74.5308,35,Hospedaje El Conquistador
111,huntingburg,US,38.2989,-86.9550,58,Quality Inn
130,kone,NC,-21.0595,164.8658,86,Pacifik Appartels
140,charters towers,AU,-20.1000,146.2667,94,No hotel found
143,jurmala,LV,56.9680,23.7704,60,Kurši
146,hamilton,US,39.1834,-84.5333,60,North Vista Manor
156,maun,BW,-19.9833,23.4167,11,Center Lodge Conference Center


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)
    

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)